# Introduction to Python and Vibration

Generally you will be creating environments in python on your computers or servers.  There the use of virtual environments and/or environment specifications can make things a lot more repeatable.

conda env create -f environment.yml

__[Conda env documentation](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file)__

python -m pip install -r requirements.txt

__[pip environment documentation](https://pip.pypa.io/en/stable/user_guide/?highlight=requirements.txt#requirements-files)__



__[Python Documentation](https://docs.python.org/3/)__

__[Software Carpentry](https://software-carpentry.org/lessons/)__

__[Built in Functions](https://docs.python.org/3/library/functions.html)__



Type Markdown and LaTex: $$a^2$$

In [ ]:
[a for a in range(5)]

In [ ]:
{'our list':[0, 1, 2, 3, 4],
'new list':[0,1]}

In [ ]:
(1,2)

In [ ]:
'strings are things'
our_list = [a for a in range(5)]
our_string = '01234'
our_tuple = (0,1,2,3,4)
our_dict={'1':1,'2':2}

In [ ]:
our_list

In [ ]:
our_string[0],our_string[-1], our_string

In [ ]:
our_list[2] = 8

In [ ]:
for a in our_dict.values():
    print("Dict Key {}".format(a))

In [ ]:
for a in our_list:
    print(a)

In [ ]:
for a in our_string:
    print(a)

In [1]:
if 10<0:
    print('Yes')
elif 10<11:
    print('ok')
else:
    print('No')

ok


In [ ]:
def first_func(x, y=2):
    '''Takes a number defaulting to 2 and adds y'''
    return x+y

In [ ]:
first_func(8, y=20)

In [ ]:
help(first_func)

__[Numpy Documentation Style Guide](https://numpydoc.readthedocs.io/en/latest/format.html)__

__[Docstring Guide](https://numpydoc.readthedocs.io/en/latest/format.html#docstring-standard)__

# Aircraft Sizing
a worked example in python 3 of chapter 3 from Aircraft Design: A Conceptual Approach 3rd edition by Daniel P. Raymer. using the ASW(Anti Submarine Warface) aircraft as an example.  Where possible no libraries not in an anaconda distribution basic install will be used.
<br/><br/>
**Mission:** <br/>Fly 1500nm (2778km) at M=0.6 carrying 10000lb (4535.924kg) of equipment and 4 crew members at 800lb (363.874kg), loiter on station for 3 hrs then return.
# Takeoff Gross Weight Estimation


$$Takeoff\; Gross\; Weight = W_0\\ 
Crew\; Weight = W_{crew}\\ 
Payload\; Weight = W_{payload}\\ 
Fuel\; Weight = W_{fuel}\\ 
Aircraft\; Empty\; Weight = W_{empty}\\ 
W_0 = W_{crew} + W_{payload} + W_{fuel} + W_{empty}$$

- Simplification of the equation
$$Empty\; Weight\; Fraction = \frac{W_{empty}}{W_0}\\ 
Fuel\; Fraction = \frac{W_{fuel}}{W_0}\\
W_0 = W_{crew} + W_{payload} + (\frac{W_{fuel}}{W_0})W_0 + (\frac{W_{empty}}{W_0})W_0$$
- Solving for $W_0$ as follows
$$ W_0 - (\frac{W_{fuel}}{W_0})W_0 - (\frac{W_{empty}}{W_0})W_0 = W_{crew} + W_{payload}$$
$$W_0 = \frac{W_{crew} + W_{payload}}{1-(\frac{W_{fuel}}{W_0})-(\frac{W_{empty}}{W_0})}$$
Since $W_{crew}$ and $W_{payload}$ are normally in the aircraft design requirments this then allows us to estimate the takeoff gross weight $W_0$ of the proposed aircraft by selecting appropriate empty weight fraction $\frac{W_{empty}}{W_0}$ and fuel fractions $\frac{W_{fuel}}{W_0}$ based on historical trends and our understanding of the aircraft role based on its design requirments.

In [ ]:
def TGW(EmptyW_F, Fuel_F, Crew_W, Payload_W):
    return (Crew_W+Payload_W)/(1-Fuel_F-EmptyW_F)
#e.g. ASW example
Crew_Weight = 800 #lb
Payload_Weight = 10000 #lb
Fuel_Fraction = 0.387
Empty_Weight_Fraction = 0.4361
print("Crew Weight %.2f lb" % (Crew_Weight))
print("Payload Weight %.2f lb" % (Payload_Weight))
print("Fuel Fraction %.4f" % (Fuel_Fraction,))
print("Empty Weight Fractiont %.4f" % (Empty_Weight_Fraction,))
TGW_tmp_1 = TGW(Empty_Weight_Fraction, Fuel_Fraction, Crew_Weight, Payload_Weight)
print("Takeoff Gross Weight W_0 %.2f lb" % (TGW_tmp_1))

# Empty Weight Estimation
Empty weight fractions are estimated based on historical trends for aircraft based on their roles and sized takeoff weight.  Raymer suggests a $5\%$ reduction should be applied if the proposed aircraft is to be of composite construction.  He suggests that there is not yet enough information to base this reduction in a study of actual aircraft.  Perhaps by now there is such a study and this notebook will perhaps be updated to reflect that in the future.
$$Historical\; metric\; = A\\
Variable\; Wing\; Sweep\; penalty\; = K_{vs}\\
K_{vs} = 1.04\; if\; variable\; sweep\; K_{vs} = 1\; otherwise\\
 \\
Weight\; Sensitivity\; Exponent\; = C\\
\frac{W_{empty}}{W_0} = A {W_0}^C K_{vs}$$

In [ ]:
def EWF(TakeoffG_W,HS_metric,WeightS_E,VWS=False,VWS_penalty=1.04):
    if VWS:
        VWS_p_factor = VWS_penalty
    else:
        VWS_p_factor = 1
        
    return HS_metric*pow(TakeoffG_W, WeightS_E)*VWS_p_factor

bomber_HS_metric = 0.93
bomber_weight_sensitivty_e = -0.07
estimated_TGW = 50000
print(EWF(estimated_TGW,bomber_HS_metric,bomber_weight_sensitivty_e))

# Fuel Fraction Estimation
To estimate the fuel fraction of the aircraft we need to understand more about how it will be used to inform us regarding the rate of fuel consumption at different points during the mission and the aerodynamics.<br/>
An aircraft mission can be broken down into multiple segments.  The weight at the start of the mission is taken as the Takeoff Gross Weight $W_0$ and for each mission segment $i$ the weight of the aircraft at the end of that segment will be $W_i$.  The weight fraction of each segment is expressed as $(\frac{W_i}{W_{i-1}})$.  In a mission with a total number of segments $x$ the weight of the aircraft at the end of the mission will be $W_x$.  The ratio $\frac{W_x}{W_0}$ can then be used to calculate the fuel fraction.  In our simple sizing study we presume no other weight change apart from fuel burn occurs during the mission.  Raymer suggests a $6\%$ allowance should be made for reserve and trapped fuel.
$$\frac{W_{fuel}}{W_0} = 1.06(1 - \frac{W_x}{W_0})$$

In [ ]:
def FF(mission_ratio,RT_Allowance=1.06):
    return RT_Allowance*(1-mission_ratio)

# Cruise fuel burn
Cruise weight fractions can be found by using the breguet range equation
$$Range = R\; in\; ft\\
Specific\; Fuel\; Consumption = C \\
Velocity = V\; in\; \frac{ft}{s}\\
Lift\; to\; Drag\; ratio\; = \frac{L}{D} \\
R = \frac{V}{C} \frac{L}{D} \ln(\frac{W_{i-1}}{W_i})$$

Which can be rearranged to give the weight fraction for that segment.
$$\frac{W_i}{W_{i-1}} = exp(\frac{-R C}{V (\frac{L}{D})})$$


In [ ]:
import numpy
def cruise_WF(Range, Specific_FC, velocity, LD_ratio):
    return numpy.exp((-1*Range*Specific_FC)/(velocity*LD_ratio))

high_bypass_turbofan_cruise_SFC = 0.0001389

# Loiter fuel burn
Loiter weight fractions can be found using the endurance equation.
$$ Endurance = E\; in\; s\\
E = \frac{\frac{L}{D}}{C}\ln(\frac{W_{i-1}}{W_i})$$
Rearranged again to put it in terms of the weight fraction we get
$$\frac{W_i}{W_{i-1}} = exp(\frac{-E C}{(\frac{L}{D})})$$

In [ ]:
def loiter_WF(Endurance, Specific_FC, LD_ratio):
    return numpy.exp((-1*Endurance*Specific_FC)/(LD_ratio))

high_bypass_turbofan_loiter_SFC = 0.0001111

# L/D Ratio esimation

Raymer suggests that during initial sizing for a jet aircraft the cruise L/D should be taken as 86.6% of the maximum L/D of the aircraft.  The loiter L/D ratio will of course be the maxium L/D as loiter is done in the most efficent configuration to ensure as much time on station as possible.

This then leavs us with the problem of estimating L/D.
Raymer suggests estimating it based on the wetted aspect ratio of the aircraft.  This then leaves us to use the conceptual sketch which has been done of the ASW aircraft concept Raymer use as an example and he gives the wetted area ratio $\frac{S_{wet}}{S_{ref}}$ as 5.5 and the wetted aspect ratio is 1.27.  Looking up these values on the figure he provides on page 26 gives us our design L/D max of 16.

In [ ]:
class AircraftMission:
    variable_stan_disp = {'name':{'description':'Name:','format':'%s'},
                         'weight_fraction':{'description':'WF:','format':'%.4f'},
                         'description':{'description':'Description:','format':'%s'},
                         'segment':{'description':'Segment:','format':'%i'}}
    def __init__(self):
        self.segments=[]
    
    def append_segment(self, seg_name, seg_WF, seg_description='None Provided'):
        no_of_segments = len(self.segments)
        self.segments.append({'segment':no_of_segments+1,'name':seg_name,'weight_fraction':seg_WF,'description':seg_description})
    
    def mission_weight_ratio(self):
        no_of_segments = len(self.segments)
        if no_of_segments <= 0:
            raise ValueError('No mission segments have been added to the mission!')
        
        mwr = 1
        for a in range(0,no_of_segments):
            mwr = mwr*self.segments[a]['weight_fraction']
        return mwr
    
    def landing_weight(self, TakeoffG_W):
        return TakeoffG_W * self.mission_weight_ratio()
    
    def display_mission(self,itp=['segment','name','weight_fraction','description'],p_dict=variable_stan_disp):
        no_of_segments = len(self.segments)
        if no_of_segments <= 0:
            raise ValueError('No mission segments have been added to the mission!')
        
        for a in range(0,no_of_segments):
            disp_list = [p_dict[item_a]['description']+p_dict[item_a]['format']+' ' for item_a in itp]
            variable_list = [self.segments[a][item_b] for item_b in itp]
            disp_string = ''.join(disp_list) % tuple(variable_list)
            print(disp_string)

In [ ]:

ASW_LD_max = 16
cruise_LD_ratio = ASW_LD_max * 0.866
loiter_LD_ratio = ASW_LD_max
cruise_velocity = 596.9 # ft/s
cruise_SFC = high_bypass_turbofan_cruise_SFC
loiter_SFC = high_bypass_turbofan_loiter_SFC
mission_loiter = 10800 # 3h in s
landing_loiter = 1200 # 20 min in s
cruise_range = 9114000 # 1500nm in ft

ASW_Mission = AircraftMission()
ASW_Mission.append_segment('Warmup and Takeoff',0.97)
ASW_Mission.append_segment('Climb',0.985)
ASW_Mission.append_segment('Cruise',cruise_WF(cruise_range,cruise_SFC,cruise_velocity,cruise_LD_ratio),seg_description='1500 nm range') 
ASW_Mission.append_segment('Loiter',loiter_WF(mission_loiter,loiter_SFC,loiter_LD_ratio),seg_description='3 hr Loiter') 
ASW_Mission.append_segment('Cruise',cruise_WF(cruise_range,cruise_SFC,cruise_velocity,cruise_LD_ratio),seg_description='1500 nm range') # 1500 nm
ASW_Mission.append_segment('Loiter',loiter_WF(landing_loiter,loiter_SFC,loiter_LD_ratio),seg_description='20 minute loiter') #20 mins
ASW_Mission.append_segment('Land',0.995)
ASW_Mission.display_mission()

MWR_tmp = ASW_Mission.mission_weight_ratio()
print("Misson Weight Ratio %.4f" % (MWR_tmp,))
FF_tmp = FF(MWR_tmp)
print("Fuel Fraction %.4f" % (FF_tmp,))
TakeoffG_W_estimate = 50000
EWF_tmp = EWF(TakeoffG_W_estimate,0.93,-0.07)
print("Empty Weight Fraction %.5f based on TGW W_0 estimate %.3f lb" % (EWF_tmp,TakeoffG_W_estimate))
TGW_tmp = TGW(EWF_tmp , FF_tmp, Crew_Weight, Payload_Weight)
print("Takeoff Gross Weight W_0 %.3f lb"  % (TGW_tmp))
LGW_tmp = ASW_Mission.landing_weight(TGW_tmp)
print("Landing Gross Weight W_x %.3f lb"  % (LGW_tmp))
fuel_burn = TGW_tmp-LGW_tmp
print("Fuel Burned %.3f lb"  % (fuel_burn))


    
def calc_WG0(WGV,Fuel_F,Crew_W, Payload_W, HS_metric,WeightS_E,VWS=False,VWS_penalty=1.04):
    EWF_calc = EWF(WGV,HS_metric,WeightS_E,VWS=False,VWS_penalty=1.04)
    return TGW(EWF_calc, Fuel_F, Crew_W, Payload_W)

def diff_calc_WG0(WGV,Fuel_F,Crew_W, Payload_W, HS_metric,WeightS_E,VWS=False,VWS_penalty=1.04):
    tmp_wgv = calc_WG0(WGV,Fuel_F,Crew_W, Payload_W, HS_metric,WeightS_E,VWS,VWS_penalty)
    result = WGV - tmp_wgv
    if result < 0:
        mag_result = result * -1
    else:
        mag_result=result
    return mag_result
    

def calc_WG0_array(WGV_list,Fuel_F,Crew_W, Payload_W, HS_metric,WeightS_E,VWS=False,VWS_penalty=1.04):
    WG0_array=[]
    EWF_array=[]
    diff_array=[]
    for a in range(0,len(WGV_list)):
        WG0_array.append(calc_WG0(WGV_list[a],Fuel_F,Crew_W, Payload_W, HS_metric,WeightS_E,VWS,VWS_penalty))
        EWF_array.append(EWF(WGV_list[a],HS_metric,WeightS_E,VWS=False,VWS_penalty=1.04))
        diff_array.append(diff_calc_WG0(WGV_list[a],Fuel_F,Crew_W, Payload_W, HS_metric,WeightS_E,VWS,VWS_penalty))
    return WG0_array, EWF_array, diff_array

Now that we have created a function for the the formulas and a way to store and interrogate mission profiles we can find the ideal $W_0$ using the iteration method Raymer shows. 

In [ ]:
import matplotlib.pyplot as plt

WGV_array = [50000]
for a in range(0,250):
    WGV_array.append(WGV_array[a]+50)
    
calc_WG0_tmp, calc_EWF_tmp, diff_array = calc_WG0_array(WGV_array,FF_tmp,Crew_Weight, Payload_Weight, bomber_HS_metric,bomber_weight_sensitivty_e)
best_guess_index = numpy.argmin(diff_array)
fig, ax = plt.subplots()
axes = [ax, ax.twinx(), ax.twinx()]
fig.subplots_adjust(right=0.75)
axes[-1].spines['right'].set_position(('axes', 1.2))
axes[0].plot(WGV_array,calc_WG0_tmp,c='b')
axes[0].set_title('Estimated vs Calculated Takeoff Gross Weight for\nASW Mission Best Guess $W_0$:%.3f lb EWF:%.4f' % (WGV_array[best_guess_index],calc_EWF_tmp[best_guess_index]))
axes[0].set_ylabel('Calculated $W_0$ (lb)',color='b')
axes[0].set_xlabel('Estimated $W_0$ (lb)')
axes[0].tick_params('y', colors='b')
axes[1].plot(WGV_array,calc_EWF_tmp,c='r')
axes[1].set_ylabel('Empty Weight Fraction',color='r')
axes[1].tick_params('y', colors='r')
axes[2].plot(WGV_array,diff_array,c='g')
axes[2].set_ylabel('|difference|',color='g')
axes[2].tick_params('y', colors='g')
plt.show()

# Optimisation
In contrast to the method expressed in Raymers book because we have expressed the formulas as python functions we can use the powerful optimisation methods available to us.  Here we use the scipy library minimize method to efficently find the ideal $W_0$ for the mission as defined.

Because python has these libraries it makes it much easier to create functions that call other functions and explore the solution space to create optimal aircraft designs.

In [ ]:
import scipy.optimize as optimize
minimum_W_0 = optimize.minimize(diff_calc_WG0,50000, args = (FF_tmp,Crew_Weight, Payload_Weight, bomber_HS_metric,bomber_weight_sensitivty_e))
min_EWF = EWF(minimum_W_0.x,0.93,-0.07)
print("Optimised Takeoff Gross Weight W_0 found %.3f lb with EWF %.4f" % (minimum_W_0.x,min_EWF))

In [ ]:
print('Difference between the Takeoff Gross Weight W_0 estimations: {} lbs'.format((WGV_array[best_guess_index] - minimum_W_0.x)[0]))
print('Difference between the Empty Weight Fraction EWF estimations: {} %'.format(100*((calc_EWF_tmp[best_guess_index] - min_EWF)[0])))

# Vibration

In [ ]:
import scipy.signal as signal
import scipy.fftpack as fftpack
from matplotlib.ticker import EngFormatter
import numpy as np

formatter_db = EngFormatter(unit='dB')
formatter_freq = EngFormatter(unit='Hz')
formatter_time = EngFormatter(unit='s')
formatter_V = EngFormatter(unit='V')

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

def decibel(val_array):
    abs_val_array = np.abs(val_array)
    array_max = np.amax(abs_val_array)
    return 20*np.log10(np.divide(abs_val_array, array_max))

In [ ]:
## Sample rate and desired cutoff frequencies (in Hz).
fs = 4000000.0
sim_gear_freq = 5400 #tri wave
sim_signal_freq = 50000 #sine wave
zoom_offset = 1000  

Vpp_sim_signal = 0.0004
Vpp_sim_gear = 0.4


lowcut = sim_signal_freq - 5000
highcut = sim_signal_freq + 5000
# Plot the frequency response for a few different orders.


# Filter a noisy signal.
T = 0.1# tim_lim=0.0003
nsamples = int(T * fs)
t = np.linspace(0, T, nsamples, endpoint=False)


x = 0.5 * Vpp_sim_gear * signal.sawtooth(2 * np.pi * sim_gear_freq * t)
x += 0.5 * Vpp_sim_signal * np.cos(2 * np.pi * sim_signal_freq * t)

freq_data =  fftpack.fftfreq(nsamples) * fs
x_fft = decibel(fftpack.fft(x))
fft_frequencies = fftpack.fftshift(freq_data)
fft_amplitudes = fftpack.fftshift(x_fft)

In [ ]:
fig, ax = plt.subplots(nrows=3,figsize=(12,6))
ax[0].plot(t,0.5 * Vpp_sim_gear * signal.sawtooth(2 * np.pi * sim_gear_freq * t),'yellow',label='Gearmesh Signal')
ax[0].yaxis.set_major_formatter(formatter_V)
ax[0].xaxis.set_major_formatter(formatter_time)
ax[0].set_xlim((0,0.0003))
ax[0].legend()
ax[1].plot(t,0.5 * Vpp_sim_signal * np.cos(2 * np.pi * sim_signal_freq * t),'red',label='MF Signal')
ax[1].yaxis.set_major_formatter(formatter_V)
ax[1].xaxis.set_major_formatter(formatter_time)
ax[1].set_xlim((0,0.0003))
ax[1].legend()
#ax[3].set_xscale('log')
ax[2].yaxis.set_major_formatter(formatter_db)
ax[2].xaxis.set_major_formatter(formatter_freq)
ax[2].plot(fft_frequencies,fft_amplitudes, color='blue')
ax[2].axvline(sim_gear_freq, linestyle=':',color='r', label='{} Tri'.format(formatter_freq(sim_gear_freq)),zorder=0)
ax[2].set_xlim((0,50000))
plt.tight_layout()
plt.show()

# Exercise

## How can you find the signal peak frequency?

## How might we simulate noise in the signal?

## Does the phase of the signals matter?  How might we explore that?

## Try to implement a short time fourier transform and plot the result

[Short Time Fourier Transform function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.stft.html#scipy.signal.stft)

## Envelope Detection

[Envelope detection using Hilbert Transform](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.hilbert.html)